In [1]:
from selenium import webdriver
from urllib.parse import quote_plus
from urllib.request import urlopen
import os
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import cv2
import openai
import ast
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import TfidfVectorizer
from skimage import io, color
from skimage.transform import resize
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from math import *

### 1. 교보문고 페이지에서 책 정보 크롤링

- 한국소설을 대상으로 일반소설, 청소년소설, 미스터리/스릴러, 판타지, 로맨스, SF/과학 부문에서<br>
  판매량 순으로 상위 200권씩 책 정보 수집

In [ ]:
book_image = []
book_info = []

# ['010101','010104','010107','010108','010111','010115'] 교보문고 각 장르별 페이지 number

for page in range(1,11):
    url = f'https://product.kyobobook.co.kr/category/KOR/010115#?page={str(page)}&type=all&per=20&sort=sel'

    # chrome 브라우저 열기
    browser = webdriver.Chrome()
    browser.implicitly_wait(3)  # 브라우저를 오픈할 때 시간간격 제공
    browser.get(url)

    # 이미지 긁어오기
    images = browser.find_elements(By.CLASS_NAME, "prod_link") # 이미지가 있는 부분 
    infos = browser.find_elements(By.CLASS_NAME, "prod_info_box") # 책 줄거리, 제목 등이 있는 부분
    book_image += images
    book_info += infos

In [ ]:
# 크롤링한 이미지 링크를 실제 jpg 파일로 다운로드 

about_img = []
idx = 1

for image in list(set(book_image)):
    html_content = image.get_attribute("outerHTML")
    soup = BeautifulSoup(html_content, 'html.parser')

    # img 태그 찾기
    img_tag = soup.find('img')

    # src 속성 추출
    try:
        book_cover = img_tag['src'] # 책 표지 이미지 
        book_title = img_tag['alt'] # 책 제목 
        img_name = 'chung' + str(idx) # 저장할 이미지 이름

        response = requests.get(book_cover)

        if response.status_code == 200:
            with open('D:\gala_end14\\book_cover\\' + img_name + '.jpg', "wb") as file:
                file.write(response.content) # 이미지 저장하기
            about_img += [[book_cover,book_title,img_name]] # 리스트에 책 정보 저장하기
            idx += 1
        else:
            print(f"이미지 다운로드 실패. 상태 코드: {response.status_code}")
    except:
        continue

In [ ]:
lst = []

for image in list(set(book_info)):
    html_content = image.get_attribute("outerHTML")
    soup = BeautifulSoup(html_content, 'html.parser')

    try:
        book_title = soup.find(attrs={'class':'prod_name'}).text.strip() # 책 이름 
        book_price = soup.find(attrs={'class':'val'}).text.strip() # 책 가격 
        about_book = soup.find(attrs={'class':'prod_introduction'}).text.strip() # 책 줄거리
        book_score = soup.find(attrs={'class':'review_klover_text font_size_xxs'}).text.strip() # 책 평점
        book_page = soup.find(attrs={'class':'point'}).text.strip() # 페이지 수
    except:
        continue
    
    lst += [[book_title, book_price, about_book, book_score,book_page,'청소년소설']]

In [ ]:
# 왜 두 개로 쪼갰을까..?

# 책의 모든 정보가 있는 DB
dic = {'book_title':np.array(lst).T[0],
       'book_price':np.array(lst).T[1],
       'about_book':np.array(lst).T[2],
       'book_score':np.array(lst).T[3],
       'book_page':np.array(lst).T[4],
       'book_genre':np.array(lst).T[5]}

# 책의 이미지와 정보만 있는 DB
dic2 = {'book_img':np.array(about_img).T[0],
        'book_title':np.array(about_img).T[1],
        'img_name':np.array(about_img).T[2]}

### 2. ChatGPT API를 활용해서 책 표지에 대한 미적 기준 관련 데이터 수집

In [ ]:
df = pd.read_csv('all_book_info.csv') # 위의 두 DB를 합친 것

In [ ]:
# ChatGPT의 답변을 기반으로 책 표지 관련 데이터 수집
# 답변을 json 형태의 txt를 반환한 적이 많아 for문 중간 중간 많이 끊겼음 

img_name = list(df['img_name'])
book_detail = list(df['about_book'])
book_genre = list(df['book_genre'])
book_title = list(df['book_title'])

for i in range(len(df)):
    print(book_title[i],i)
    img = cv2.imread(f'D:\gala_end14\\book_cover\\{img_name[i]}.jpg')
    atmosphere = ['신비로운', '따뜻한', '몽환적인', '차분한', '화사한', 
                '싱그러운', '감성적인', '컬러풀한', '미니멀리즘', '고급스러운']

    content = f'''
    너는 책 표지의 아름다움을 평가하는 평론가야. 
    책 이미지와 정보를 기반으로 평가해줘.
    평가 기준은 5가지야.
    1. 조화 (Harmony) : 평가 내용: 색상, 글꼴, 이미지, 레이아웃 등의 요소들이 서로 잘 어울리는지,
    2. 독창성 (Originality) : 평가 내용: 표지 디자인이 얼마나 참신하고 창의적인지,
    3. 가독성 (Readability) : 평가 내용: 제목, 저자 이름, 부제 등 텍스트가 쉽게 읽히는지,
    4. 심미성 (Aesthetics) : 평가 내용: 표지 자체가 얼마나 시각적으로 아름답고 매력적인지,
    5. 직관성 (Intuitiveness) : 평가 내용: 표지가 책의 내용이나 주제를 얼마나 직관적으로 전달하는지

    책 줄거리는 {book_detail[i]}이고, 책의 종류는 {book_genre[i]}, 이미지는 {img}야.

    -> 해당 기준을 기반으로 책에 대한 정보까지 반영해서 전체 이미지 점수를 0~100까지 매겨줘. 
    책에 대한 점수가 비슷하지 않도록 다양한 숫자로 반환해줘. 
    결과는 
        '조화': ??,
        '독창성': ??,
        '가독성': ??,
        '심미성': ??,
        '직관성': ??,
        '키워드': "['?', '?', '?']" 이런식으로 딕셔너리 형태로 반환하되 특수 문자 없이 문자열로 반환해줘
    설명은 필요없어. 키워드는 해당 책 표지에 대해 알맞는 분위기를 {atmosphere}안에서만 단어 3개만 선정해줘. [?, ?, ?] 이런식으로
    리스트를 반환해서 위의 딕셔너리처럼 만들어줘. 답변은 정해진 양식을 벗어나면 안돼
    '''

    openai.api_key = "api_key"
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": content},
        ],
    )

    data_dict = ast.literal_eval(response.choices[0].message.content)
    data_dict['book_title'] = book_title[i] # 책 제목을 기반으로 기존 DB와 합치기 위해
    
    img_evaluation = pd.concat([img_evaluation, pd.DataFrame([data_dict])]) # 책 표지 관련 DB 생성 후 계속 덧붙임

### 3. 줄거리 내 핵심 키워드 파악을 위한 TF-IDF 분석

In [ ]:
kiwi = Kiwi()

In [ ]:
# 불용어 제거
stop_word_ko = pd.read_table('ko_stopwords.txt') # 출처 : https://github.com/stopwords-iso/stopwords-ko
stopwords = list(stop_word_ko['stopwords'])

In [ ]:
# 명사 추출 함수
def extract_noun(text):
    result = kiwi.tokenize(text)
    for token in result:
        if token.len > 1 and token.tag in ['NNG', 'NNP'] and token.form not in stopwords: 
          # 단어의 길이가 1 이상인 명사만 추출하겠다는 의미
          yield token.form

In [ ]:
cnt = 0
data_list = list(df['about_book']) # 책 줄거리 리스트

data_noun_list = [] # 책 줄거리에서 명사만 추출한 리스트 생성
for data in data_list:
  data_noun_list.append(list(extract_noun(data)))

In [ ]:
# 9265는 책 줄거리 전체에서 추출된 총 명사 단어 개수
tv = TfidfVectorizer(max_features=9265, tokenizer=extract_noun,min_df=2,analyzer='word')
tf_idf = tv.fit_transform(data_list)

In [ ]:
feature_names = tv.get_feature_names_out()

l = []

# 각 문서별 상위 3개 단어 추출
top_n = 3
top_keywords_per_doc = []

for i in range(tf_idf.shape[0]):
    row = tf_idf[i, :].toarray().flatten()  # 각 문서의 TF-IDF 벡터
    top_indices = row.argsort()[-top_n:][::-1]  # 상위 3개 단어의 인덱스
    top_keywords = [(feature_names[idx], row[idx]) for idx in top_indices] # 상위 3개의 중요 단어 리스트
    top_keywords_per_doc.append(top_keywords)

df['keyword'] = l # DB에 컬럼 추가

### 4. 책 표지의 핵심 컬러 3가지 추출하기

In [ ]:
color_list = {(255, 0, 0):'Red',
              (255,165,0):'Orange',
              (255,255,0):'Yellow',
              (0, 128, 0):'Green',
              (0, 0, 255):'Blue',
              (76,0,153):'Purple',
              (255, 102, 255):'Pink',
              (0, 0, 0):'Black',
              (255, 255, 255):'White',
              (255, 229, 204):'Beige',
              (153,76,0):'Brown'} # R, G, B의 순서

def compare_color(palette):
    x = 0
    color = 'red'

    for i in color_list.keys():
        color_diff = sqrt(sum(pow(a - b, 2) for a, b in zip(i,palette))) # 유클리드 거리
        if list(i) == [255, 0, 0]: # 만약 i가 RGB로 했을 때 빨간색이라면 
            x = color_diff 
        else:
            if x > color_diff: # 만약 컬러 비교 중 더 가까운 컬러가 있다면 
                x = color_diff # 컬러의 값과 
                color = color_list[i] # 컬러 이름을 바꾸기

    return color

In [ ]:
img_name_df = []
colors_df = []

for i in df['img_name']:
    image = io.imread(f'D:\gala_end14\\book_cover\{i}.jpg') # 이미지 불러오기
    image_resized = resize(image, (100, 100))  # 속도 향상을 위해 크기 축소

    # 2D 데이터 변환
    pixels = image_resized.reshape(-1, 3)

    # K-Means 적용
    kmeans = KMeans(n_clusters=3).fit(pixels)
    palettes = kmeans.cluster_centers_

    colors = list(set([compare_color([int(i*255) for i in palette]) for palette in palettes]))
    # 0~1사이로 정규화된 값을 RGB값으로 변환하기 위해 255를 곱함 

    img_name_df += [i]
    colors_df += [colors]

In [ ]:
dic = {'img_name':img_name_df,
       'color':colors_df}

df = pd.merge(df, pd.DataFrame(dic), on='img_name') # 이미지 이름을 기반으로 기존 DB와 컬러 합치기

### 5. 추천 시스템 알고리즘 구현

1. 장르 : 이걸로 한 번 거르기 
2. 책 표지 색상 : 해당 사항 존재 시 컬러 개수만큼 +100점
3. 책 표지의 분위기 : 분위기에 따라 +50점
4. 인내심 단계 : 단계에 따라 인내심이 낮은 사람에게는 얇은 책을 추천해주려고 함

In [ ]:
import pandas as pd 
import numpy as np

def patience_classification(reco, patience): # 인내심이 높지 않은 경우 두꺼운 책에 가중치를 덜 부여하는 코드
    if patience == 'lv2': # 인내심이 덜 높음
        book_page = reco[reco['book_page'] >= 900] 
        reco.loc[book_page.index, 'art_reputation'] -= 20 # 900페이지 이상인 경우 평가 점수에서 -20점

    elif patience == 'lv3': # 인내심이 안 높음  
        book_page = reco[reco['book_page'] >= 500]
        reco.loc[book_page.index, 'art_reputation'] -= 20 # 500페이지 이상인 경우 평가 점수에서 -20점

def color_add_weight(reco, color_palettes): # 사용자의 선호 색상과 일치하는 색상 개수만큼 평가점수 +100점
    # 책 표지의 색상이 사용자가 선정한 것과 많이 다르면 사용자 기준에서 해당 서비스의 신뢰도가 낮게 평가될 것을 
    # 우려하여 색상에 가장 높은 가중치를 부여함
    for co in list(color_palettes):
        color_contain = reco[reco['color'].str.contains(co.capitalize())]
        reco.loc[color_contain.index, 'art_reputation'] += 100

def atmo_add_weight(reco,novel_atmo): # 사용자의 선호 분위기과 일치하는 분위기 개수만큼 평가점수 +50점
    for atmo in list(novel_atmo):
        atmo_contain = reco[reco['atmosphere'].str.contains(atmo.capitalize())]
        reco.loc[atmo_contain.index, 'art_reputation'] += 50

def recommend_book(user_input):
    df = pd.read_csv('recommend_data.csv')
    reco = df[df['book_genre'] == user_input['novel_genre']] # 사용자가 선호하는 장르의 책 목록만 선정 

    color_add_weight(reco, user_input['color_palettes'])
    atmo_add_weight(reco, user_input['novel_atmo'])
    
    patience_classification(reco, user_input['patience'])

    result = reco[reco['art_reputation'] == reco['art_reputation'].max()]
    # 평가 점수가 가장 높은 책 정보 반환

    return result